In [1]:
import torch
from torch import nn

class MLP(nn.Module):
    # 声明带有模型参数的层,这里声明了两个全连接层
    def __init__(self, **kwargs):
        # 调用 MLP 父类 Block 的构造函数来进行必要的初始化。这样在构造实例时还可以指定其他函数 
        # 参数,如“模型参数的访问、初始化和共享” 一节将介绍的模型参数params
        super(MLP, self).__init__(**kwargs)
        self.hidden = nn.Linear(28 * 28, 256) # 隐藏层
        self.act = nn.ReLU()
        self.output = nn.Linear(256, 10) # 输出层
        
    # 定义模型的前向计算,即如何根据输入x计算返回所需要的模型输出
    def forward(self, x):
        a = self.act(self.hidden(x))
        return self.output(a)

## 实例化 net 变量

In [5]:
X = torch.rand(2, 28 * 28)
net = MLP()
print(net)
print(X)
net(X)

MLP(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)
tensor([[0.3339, 0.6754, 0.3144,  ..., 0.8471, 0.2965, 0.9148],
        [0.6417, 0.7926, 0.7024,  ..., 0.9714, 0.7874, 0.8434]])


tensor([[-0.0315,  0.0535, -0.1689, -0.0216, -0.0975, -0.1673,  0.1781, -0.1765,
          0.0267,  0.0674],
        [-0.0280,  0.0417,  0.0613, -0.1011, -0.2433,  0.0280,  0.2121, -0.1117,
         -0.1569, -0.0076]], grad_fn=<AddmmBackward0>)

In [6]:
print(X)

tensor([[0.3339, 0.6754, 0.3144,  ..., 0.8471, 0.2965, 0.9148],
        [0.6417, 0.7926, 0.7024,  ..., 0.9714, 0.7874, 0.8434]])


## Sequential 类

In [9]:
from collections import OrderedDict
class MySequential(nn.Module):
    def __init__(self, *args):
        super(MySequential, self).__init__()
        if len(args) == 1 and isinstance(args[0], OrderedDict): # 如果传入的是一个OrderedDict
            for key, module in args[0].items():
                self.add_module(key, module) # add_module方法会将module添加进self._modules(一个OrderedDict)
            else: # 传入的是某个Module
                for idx, module in enumerate(args):
                    self.add_module(str(idx), module)
    
    def forward(self, input):
        # self._modules 返回一个 OrderedDict
        for module in self._modules.values():
            input = module(input)
        return input

In [11]:
net = nn.Sequential(
    nn.Linear(28 * 28, 256),
    nn.ReLU(),
    nn.Linear(256, 10),
)
print(net)
net(X)

Sequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[ 0.0567, -0.0531, -0.2672, -0.0322, -0.2290, -0.0737,  0.1635, -0.0397,
         -0.0963, -0.0412],
        [ 0.2016,  0.2351, -0.1288, -0.1535, -0.1899, -0.1304,  0.1921,  0.0831,
         -0.1079,  0.0674]], grad_fn=<AddmmBackward0>)

## ModuleList 类

In [13]:
net = nn.ModuleList([
    nn.Linear(784, 256),
    nn.ReLU()
])
net.append(nn.Linear(256, 10)) # 类似 List 操作
print(net[-1])
print(net)

Linear(in_features=256, out_features=10, bias=True)
ModuleList(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


## ModuleDict 类

In [16]:
net = nn.ModuleDict({
    'linear': nn.Linear(784, 256),
    'relu': nn.ReLU(),
})
net['output'] = nn.Linear(256, 10)
print(net['linear'])
print(net)

Linear(in_features=784, out_features=256, bias=True)
ModuleDict(
  (linear): Linear(in_features=784, out_features=256, bias=True)
  (relu): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


# 构造复杂的模型

In [17]:
class FancyMLP(nn.Module):
    def __init__(self, **kwargs):
        super(FancyMLP, self).__init__(**kwargs)
        
        self.rand_weight = torch.rand((20, 20), requires_grad=False) # 不可训练参数
        self.linear = nn.Linear(20, 20)
        
    def forward(self, x):
        x = self.linear(x)
        
        # 使用创建的常数参数,以及nn.functional中的relu函数和mm函数
        x = nn.functional.relu(torch.mm(x, self.rand_weight.data) + 1)
        
        # 复用全连接层。等价于两个全连接层共享参数
        x = self.linear(x)
        
        # 控制流,这里我们需要调用item函数来返回标量进行比较
        while x.norm().item > 1:
            x /= 2
        if x.norm().item < 0.8:
            x *= 10
        return x.sum()

In [18]:
X = torch.rand(2, 20)
net = FancyMLP()
print(net)
print(X)

FancyMLP(
  (linear): Linear(in_features=20, out_features=20, bias=True)
)
tensor([[0.4840, 0.8566, 0.4564, 0.9277, 0.0792, 0.7144, 0.6244, 0.7763, 0.6368,
         0.7236, 0.7780, 0.8287, 0.9130, 0.1909, 0.8145, 0.1106, 0.7212, 0.5668,
         0.3767, 0.4072],
        [0.6896, 0.4722, 0.0433, 0.0655, 0.4583, 0.8127, 0.8067, 0.0463, 0.7229,
         0.0962, 0.3270, 0.1076, 0.9315, 0.4078, 0.5247, 0.2931, 0.7087, 0.1210,
         0.4497, 0.0967]])
